In [11]:
import re
import pandas as pd
import numpy as np
import simplemma
from stop_words import get_stop_words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("ggplot")
import xgboost as xgb

In [2]:
stop_words = get_stop_words('uk')

In [3]:
#попередня обробка тексту
def ClearText(text):
    #переведення до нижнього регістру всіх слів
    cleartext = text.lower()
    #print(cleartext)
    #прибирання пустих рядків та розрив рядків
    cleartext = re.sub('\-\s\r\n\s{1,}|\-\s\r\n|\r\n', '', cleartext) 
    #залишаємо лише слова, прибираємо пунктуацію та числа
    cleartext = re.sub('[.,:;_%©?*,!@#$%^&()\d]|[+=]|[[]|[]]|[/]|"|\s{2,}|-', ' ', cleartext) #deleting symbols  
    #cleartext = cleartext.translate(remove_digits)
    cleartext = cleartext.replace("\\", "")
    cleartext = cleartext.rstrip()
    #прибираємо зайві пробіи
    cleartext = re.sub(" +", " ", cleartext)
    #ділимо речення на список слів, розбиваємо по пробілам
    cleartext = re.split(" ", cleartext)
    #прибираємо стопслова
    cleartext = [word for word in cleartext if word not in stop_words]
    #прибираємо слова, довжина який менше 3 букв
    cleartext = [word for word in cleartext if len(word) > 3]
    #лематизація слів
    cleartext = [simplemma.lemmatize(word, lang='uk') for word in cleartext]
    cleartext = [word.lower() for word in cleartext ]
    return ' '.join(cleartext)

In [5]:
df = pd.read_excel('C:/Users/helen/OneDrive/Рабочий стол/пары/NLP_all.xlsx')
df = df.sample(frac=1) 
df['ClearText'] = df.apply(lambda x: ClearText(x['Comment']), axis=1)
df

,Comment,Category,ClearText
846,"Боже, яка краса!!!! Так надихаюче!!!! Чарівна ...",Not_Toxic,бог краса надихаюче чарівний сім‘я прекрасний ...
185,"Як гадаєте, чому люди бояться висвітлювати в с...",Not_Toxic,гадати боятися висвітлювати соцмережах реальний
326,"генії,в країні йде війна 8 років а вони відпра...",Toxic,геній країна війна відправляти пітер
11,Україна була! Україна є! Україна буде! Слава У...,Toxic,україна україна україна слава україна герой сл...
1136,"ЛАХУДРА БРЕХЛИВА.\nОДНА З ТИХ ЖІНОК, ЯКІ ЗАРАД...",Toxic,лахудра брехливий \nодна жінка заради чоловік ...
...,...,...,...
682,Це воля !!! Воля душі і тіла !!! Хто не жив у ...,Not_Toxic,воля воля душ тіло село знати ідея хороший
625,Блондин повернувся до України?,Not_Toxic,блондин повернутися україна
630,Ну той мужик у рожевому халаті 200 кг мінімум,Toxic,мужик рожевий халат мінімум
1032,Подивіться Дніпро.,Not_Toxic,подивитися дніпро


In [6]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(df['ClearText'])
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [7]:
df['TF-IDF'] = list(X_train_tfidf.toarray())

In [8]:
df['category1'] = df.apply(lambda x: 0 if x['Category'] == 'Not_Toxic' else 1, axis=1)

In [9]:
df_train = df.iloc [:round(df.shape[0]*0.75)]
df_test = df.iloc [round(df.shape[0]*0.75):]

# booster='gbtree'

In [13]:
model = xgb.XGBClassifier()
model.fit(list(df_train['TF-IDF']), list(df_train['category1']))
print(model)
# make predictions for test data
y_pred = model.predict(list(df_test['TF-IDF']))
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(list(df_test['category1']), predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

[15:36:48] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
Accuracy: 65.29%


# booster='gblinear'

In [14]:
model = xgb.XGBClassifier(booster='gblinear')
model.fit(list(df_train['TF-IDF']), list(df_train['category1']))
print(model)
y_pred = model.predict(list(df_test['TF-IDF']))
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(list(df_test['category1']), predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

C:\Users\helen\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:37:12] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier(base_score=0.5, booster='gblinear', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              enable_categorical=False, gamma=None, gpu_id=-1,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.5, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=8, num_parallel_tree=None,
              predictor=None, random_state=0, reg_alpha=0, reg_lambda=0,
              scale_pos_weight=1, subsample=None, tree_method=None,
              validate_parameters=1, verbosity=None)
Acc

# booster='dart'

In [15]:
model = xgb.XGBClassifier(booster='dart')
model.fit(list(df_train['TF-IDF']), list(df_train['category1']))
print(model)
y_pred = model.predict(list(df_test['TF-IDF']))
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(list(df_test['category1']), predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

C:\Users\helen\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:37:20] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier(base_score=0.5, booster='dart', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
Accuracy: 65.29%


# Best accuracy

In [74]:
model = xgb.XGBClassifier(base_score=0.5, booster='gblinear', learning_rate=0.1, scale_pos_weight=1,
               n_estimators=250, n_jobs=15)
model.fit(list(df_train['TF-IDF']), list(df_train['category1']))
print(model)
# make predictions for test data
y_pred = model.predict(list(df_test['TF-IDF']))
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(list(df_test['category1']), predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

C:\Users\helen\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:02:15] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier(base_score=0.5, booster='gblinear', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              enable_categorical=False, gamma=None, gpu_id=-1,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=250, n_jobs=15, num_parallel_tree=None,
              predictor=None, random_state=0, reg_alpha=0, reg_lambda=0,
              scale_pos_weight=1, subsample=None, tree_method=None,
              validate_parameters=1, verbosity=None)
Ac

# XGBRegressor

In [16]:
model = xgb.XGBRegressor()
model.fit(list(df_train['TF-IDF']), list(df_train['category1']))

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [17]:
expected_y  = list(df_test['category1'])
predicted_y = model.predict(list(df_test['TF-IDF']))

In [18]:
predictions_reg = [round(value) for value in predicted_y]

In [19]:
accuracy = accuracy_score(expected_y, predictions_reg)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 63.92%
